![Colegio Bourbaki](./Images/Bourbaki.png)

## Procesamiento de Lenguaje Natural

### Contexto

El objetivo de este notebook es hacer una demostración de la creación de chatbots estilo ChatGPT con conocimiento de datos específicos.

En primer lugar, enseñaremos cómo conectar con el API de OpenAI para utilizar GPT-3.5 Turbo, el modelo que alimenta a la versión abierta de ChatGPT, desde código.

Después, veremos cómo podemos introducir material a la base de conocimiento del chatbot, para así obtener respuestas más personalizadas.

### Librerías

In [1]:
# NLP Chatbots
#!pip install openai langchain duckdb unstructured chromadb tiktoken
import openai
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import VectorDBQA
from langchain.chat_models import ChatOpenAI

#Utils
import os
from dotenv import load_dotenv #!pip install python-dotenv
from pdfminer.high_level import extract_text #!pip install pdfminer.six

In [2]:
import warnings
warnings.filterwarnings("ignore")

Conectamos con el API GPT-3.5 por medio de una llave privada a cada usuario.

In [3]:
load_dotenv() # This method loads the variables from .env into the environment

True

In [4]:
#api_key = os.environ['CHATGPT_API_KEY']
api_key = os.getenv("CHATGPT_API_KEY") # This method loads the variables from .env into the environment with dotenv
if api_key is None:
    raise ValueError("API key not found. Please set the CHATGPT_API_KEY environment variable.")

In [5]:
client = openai.OpenAI(
    api_key=api_key,
)

Una vez hecha la conexión con GPT-3.5 podemos aprovechar las capacidades conversacionales de ChatGPT. 

Desde este punto ya es posible integrar asistentes inteligentes a aplicaciones y sistemas.

In [6]:
prompt = "¡Hola! ¿Por qué el cielo es azul?"

In [7]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
    #model="gpt-4",
)

In [8]:
print(chat_completion.choices[0].message.content)

¡Hola! El cielo se ve azul debido a un fenómeno llamado dispersión de la luz. La luz del sol está compuesta por diferentes colores, cada uno con una longitud de onda diferente. Cuando la luz del sol atraviesa la atmósfera de la Tierra, las moléculas de aire dispersan los colores de luz de manera diferente. Los colores más cortos, como el azul y el violeta, se dispersan más que los colores más largos, como el rojo y el naranja. Esto significa que vemos principalmente el color azul dispersado en el cielo durante el día, lo que nos da esa apariencia azul. Por la noche, cuando el sol no está presente, el cielo se ve oscuro porque la luz del sol ya no la ilumina.


Podemos lograr mejores respuestas del modelo si modificamos el atributo **system**.

In [9]:
sistema = "Eres un asistente de poetas, habilidoso en explicar conceptos complejos de programación creativamente."
usuario = "Compón un poema que explique el concepto de recursión en programación."

chat_completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": sistema},
    {"role": "user", "content": usuario}
  ]
)

In [10]:
print(chat_completion.choices[0].message.content)

En el vasto lenguaje de código enredado,
donde la lógica y la creatividad se han encontrado,
existe un concepto, brillante y profundo,
que se llama recursión, un poderoso mundo.

Imagina un bucle, tan intrincado,
que no es lineal, sino un tanto enredado,
una función que se llama a sí misma,
volviendo al principio sin ninguna envidia.

Como un laberinto de caminos sin fin,
la recursión sigue patrones, sin limitación,
dividiendo problemas en tareas más pequeñas,
resolviendo cada una con maestría y destreza.

En cada iteración, se acerca a la solución,
rompiendo problemas con gran resolución,
hasta que se cumpla una condición de salida,
y se desenmarañen los nudos, sin prisa.

Es como un espejo que se refleja a sí mismo,
creando copias de sí, sin pensar en sí mismo,
un eco infinito de instrucciones recursivas,
hasta que el problema original se desvanece.

Aunque puede ser difícil de dominar,
la recursión es un arte, un modo de pensar,
un enfoque poderoso y elegante,
que a problemas comple

### Creación de un asistente especializado

A su vez, podemos aprovechar aún más las capacidades de loss LLM haciendo una especie de fine-tuning. La idea consiste en alimentar al modelos con documentos propios para así lograr respuestas informadas sobre ellos.

Esto es posible en GPT-3.5 a través de los encajes y la generación de una base de datos vectorizada.

Primero, extraemos texto desde archivos pdfs...


... y extramos el texto de un pdf para después guardarlo en .txt.

In [11]:
def extractor_texto(ruta):
    # Assume extract_text is a function defined elsewhere to extract text from the given path
    txt = extract_text(ruta)
    
    # Clean and format the text
    replacements = {
        '\n\n\x0c': ' ',  # Remove specific pattern
        '...': ' ',       # Replace ellipses with space
        '\n': ' ',        # Replace newline characters with space
        '  ': ' ',        # Replace double spaces with single space
        "\f": ' ',        # Remove form feed characters
        "-": ' '          # Replace hyphens with space
    }
    
    # Apply replacements
    for old, new in replacements.items():
        txt = txt.replace(old, new)
    
    # Split into paragraphs and filter based on conditions
    paragraphs = txt.split('\n\n')
    paragraphs = [paragraph.strip() for paragraph in paragraphs if len(paragraph.strip()) > 30]
    
    # Join the cleaned paragraphs
    cleaned_text = '\n'.join(paragraphs)
    
    # Write the cleaned text to a file, appending '.txt' to the original path and using utf-16 encoding
    with open(ruta + '.txt', "w", encoding="utf-16") as archivo:
        archivo.write(cleaned_text)
    
    # Optionally, return the cleaned text if needed
    return cleaned_text


In [12]:
ruta = './Data/Feynman1982_Article_SimulatingPhysicsWithComputers.pdf'

In [13]:
paper = extractor_texto(ruta)

In [14]:
with open("./Data/output.txt", "w") as text_file:
    print(paper, file=text_file)

In [15]:
loader = UnstructuredFileLoader('./Data/output.txt')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key = api_key)

db = Chroma.from_documents(texts, embeddings)

Created a chunk of size 1460, which is longer than the specified 1000
Created a chunk of size 1493, which is longer than the specified 1000
Created a chunk of size 1480, which is longer than the specified 1000
Created a chunk of size 1385, which is longer than the specified 1000
Created a chunk of size 1425, which is longer than the specified 1000
Created a chunk of size 1405, which is longer than the specified 1000
Created a chunk of size 1348, which is longer than the specified 1000
Created a chunk of size 1484, which is longer than the specified 1000
Created a chunk of size 1354, which is longer than the specified 1000
Created a chunk of size 1467, which is longer than the specified 1000
Created a chunk of size 1318, which is longer than the specified 1000
Created a chunk of size 1437, which is longer than the specified 1000
Created a chunk of size 1490, which is longer than the specified 1000
Created a chunk of size 1315, which is longer than the specified 1000
Created a chunk of s

En seguida, cargamos el documento a la base de datos.

In [16]:
qa = VectorDBQA.from_chain_type(llm=ChatOpenAI(openai_api_key=api_key), chain_type="stuff", vectorstore=db, k=1)

Este modelo ahora se puede utilizar como un ChatGPT con conocimiento especializado:

In [17]:
query = "What the document is about?"
qa.run(query)

"The document is about Richard P. Feynman's keynote speech on the topic of simulating physics with computers. Feynman discusses the inspiration for his interest in the subject and the potential for learning about both computers and physics through simulation."

Y es posible obtener respuestas muy específicas y personalizadas sobre nuestros documentos:

In [18]:
query = "How photons are polarized?"
qa.run(query)

'Photons can be polarized through a process called polarization. Polarization refers to the orientation of the electric field vector of a light wave. When light is unpolarized, the electric field vector can vibrate in any direction perpendicular to the direction of propagation. However, when light passes through certain materials or undergoes certain interactions, the electric field vector becomes restricted to a specific orientation, resulting in polarized light. This can be achieved through various methods, such as using polarizers or passing light through certain crystals.'

In [19]:
query = "Expand the concept of negative probabilities"
qa.run(query)

'In traditional probability theory, probabilities are always non-negative values between 0 and 1. However, in certain contexts, such as quantum mechanics, the concept of negative probabilities has been explored.\n\nNegative probabilities can be thought of as a mathematical tool to describe situations where the probability of an event occurring is less than zero. This may seem counterintuitive, as probabilities are typically associated with the likelihood of something happening. However, in quantum mechanics, negative probabilities can arise when dealing with certain quantum states or processes.\n\nOne interpretation of negative probabilities in quantum mechanics is that they represent a type of "anti-probability" or "anti-chance." These negative probabilities can be used to describe phenomena such as quantum superposition, where a particle can exist in multiple states simultaneously. In this case, the negative probabilities can be seen as indicating the interference or cancellation of 

**Conexión con la API de OpenAI:**

1) ¿Qué información necesitas para autenticarte y realizar peticiones al API de OpenAI?
2) ¿Cuál es el propósito de utilizar una clave API en la conexión con OpenAI y cómo se debe proteger?
3) ¿Qué diferencias hay entre los distintos modelos de OpenAI y cómo elegirías uno para tu aplicación específica?

**Uso de GPT-3.5 Turbo:**

4) ¿Qué ventajas ofrece GPT-3.5 Turbo para la creación de chatbots comparado con versiones anteriores?
5) ¿Cómo se formulan las peticiones al modelo GPT-3.5 Turbo para generar respuestas coherentes y relevantes?
6) ¿Qué limitaciones tiene el modelo GPT-3.5 Turbo y cómo puedes mitigarlas?

**Introducción de Material a la Base de Conocimiento:**

7) ¿Cómo puedes personalizar las respuestas de GPT-3.5 Turbo utilizando información específica?
8) ¿Cuál es la importancia de la relevancia y precisión del material que se introduce en la base de conocimientos del bot?
9) ¿Qué estrategias se pueden utilizar para mantener actualizada la base de conocimientos del chatbot?

**Personalización y Respuestas del Chatbot:**

10) ¿De qué manera se puede ajustar el tono o el estilo de las respuestas que genera GPT-3.5 Turbo?
11) ¿Cómo afecta el contexto proporcionado a las respuestas generadas por el chatbot?
    Describe un método para evaluar la precisión y utilidad de las respuestas del chatbot.

**Problemas Éticos y de Privacidad:**

12) ¿Cuáles son las consideraciones éticas al utilizar modelos de lenguaje generativos como GPT-3.5 Turbo en un chatbot?
13) ¿Cómo debería manejar un chatbot las solicitudes de datos personales o sensibles de los usuarios?
14) ¿Qué medidas se pueden tomar para garantizar la privacidad y la seguridad de los usuarios al interactuar con un chatbot?

![Lenguaje Matemático](./Images/Matematicas.png)

![Contacto](./Images/Contacto.png)